In [5]:
import geopandas as gpd
import sqlite3

import numpy as np
import pandas as pd
from shapely import wkt
import pyogrio
from tqdm import tqdm

data_raw = "C:/Users/eleonore.kong/Documents/InSAR/DATA/wavelet_data.db"
id_geom = "C:/Users/eleonore.kong/Documents/InSAR/DATA/EGMS_pid_geometry.db"
hex_path = "C:/Users/eleonore.kong/Documents/InSAR/DATA/grid.gpkg"
geopackage_path = "C:/Users/eleonore.kong/Documents/InSAR/DATA/wavelet_egms_data.gpkg"
# grids = ['grid_1km', 'grid_500m']

In [6]:
# Connect to the SQLite database
conn = sqlite3.connect(data_raw)
conn_pid = sqlite3.connect(id_geom)

query = "SELECT name FROM sqlite_master WHERE type='table';"
query_pid = "SELECT * FROM points"
# 
# Execute the query and get the table names
tables = pd.read_sql_query(query, conn)
geom = pd.read_sql_query(query_pid, conn_pid)
conn_pid.close()

# hex = [gpd.read_file(hex_path, layer=layer, columns=['id', 'geom']) for layer in grids]
# hex = dict(zip(grids, hex))
tables_list = tables['name'].tolist()
tables_list = [table for table in tables_list if 'full' in table]
print(tables_list)

['wavelet_full_EGMS_L3_E32N28_100km_U', 'wavelet_full_EGMS_L3_E34N25_100km_U', 'wavelet_full_EGMS_L3_E34N22_100km_U', 'wavelet_full_EGMS_L3_E34N26_100km_U', 'wavelet_full_EGMS_L3_E36N25_100km_U', 'wavelet_full_EGMS_L3_E35N27_100km_U', 'wavelet_full_EGMS_L3_E33N27_100km_U', 'wavelet_full_EGMS_L3_E34N30_100km_U', 'wavelet_full_EGMS_L3_E35N29_100km_U', 'wavelet_full_EGMS_L3_E34N29_100km_U', 'wavelet_full_EGMS_L3_E33N28_100km_U', 'wavelet_full_EGMS_L3_E34N23_100km_U', 'wavelet_full_EGMS_L3_E34N27_100km_U', 'wavelet_full_EGMS_L3_E35N24_100km_U', 'wavelet_full_EGMS_L3_E35N28_100km_U', 'wavelet_full_EGMS_L3_E35N25_100km_U', 'wavelet_full_EGMS_L3_E36N23_100km_U', 'wavelet_full_EGMS_L3_E35N26_100km_U', 'wavelet_full_EGMS_L3_E35N22_100km_U', 'wavelet_full_EGMS_L3_E33N29_100km_U', 'wavelet_full_EGMS_L3_E34N28_100km_U', 'wavelet_full_EGMS_L3_E33N23_100km_U', 'wavelet_full_EGMS_L3_E33N22_100km_U', 'wavelet_full_EGMS_L3_E36N21_100km_U', 'wavelet_full_EGMS_L3_E36N22_100km_U', 'wavelet_full_EGMS_L3_E3

In [7]:
# Query the table
print('Found {} tables'.format(len(tables_list)))
i=0

for table_name in tqdm(tables_list):

    query = "SELECT * FROM " + table_name 

    # Read the data into a Pandas DataFrame
    df = pd.read_sql_query(query, conn)
    # df.drop('geometry', axis=1, inplace=True)
    
    # pid_list = df['pid'].to_list()
    # pid_geom = geom[geom['pid'].isin(pid_list)].copy()
    # pid_geom['geometry'] = pid_geom['geometry'].apply(wkt.loads)

    # df_geom = df.merge(pid_geom, how='left', on='pid')

    # Convert to a GeoDataFrame
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326').to_crs(epsg='2154')
    gdf = gdf.reset_index()

    if i == 0:
        pyogrio.write_dataframe(gdf, geopackage_path, layer="data", driver="GPKG")
    else:
        pyogrio.write_dataframe(gdf, geopackage_path, layer="data", driver="GPKG", append=True)
        
    i+=1
    
# Close the connection
conn.close()


Found 81 tables


100%|██████████| 81/81 [47:17<00:00, 35.03s/it]   
